In [ ]:
from astroquery.mast import Observations

In [ ]:
obslist = Observations.query_object("HD 21749", radius="0.02 deg")
obslist

In [ ]:
dp = Observations.get_product_list(obslist[obslist['obs_collection'] == 'TESS'])
dp

In [ ]:
is_lightcurve = (dp['dataproduct_type'] == 'timeseries') & (dp['productSubGroupDescription'] == 'LC')

In [ ]:
manifest = Observations.download_products(dp[is_lightcurve])
manifest

In [ ]:
from astropy.timeseries import TimeSeries

In [ ]:
ts = TimeSeries.read(manifest['Local Path'][2], format='tess.fits')
ts

In [ ]:
import matplotlib.pyplot as plt
plt.plot(ts.time.jd, ts['sap_flux'], 'k.', markersize=1)
plt.xlabel('Julian Date')
plt.ylabel('SAP Flux (e-/s)')

In [ ]:
from aas_timeseries import InteractiveTimeSeriesFigure

In [ ]:
fig = InteractiveTimeSeriesFigure()
fig.xlabel = 'Time (UTC)'
fig.ylabel = 'Flux (electron/s)'

In [ ]:
markers = fig.add_markers(time_series=ts, column='sap_flux', label='SAP Flux')

In [ ]:
fig.preview_interactive()